### This is the notebook for the batch offline deployment
Additionally, the model will be used to score. 

In [42]:
import pickle
import os
import pandas as pd
import uuid


from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

import mlflow

In [43]:
year = 2021
month = 2
taxi_type = 'green'

INPUT_FILE = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
#INPUT_FILE = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet'
OUTPUT_FILE = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [51]:
RUN_ID = os.getenv('RUN_ID', '250694a4d07240ec8858323e6590c050')
logged_model = f's3://mlopsbatch/1/{RUN_ID}/artifacts/model'

#model = mlflow.pyfunc.load_model(logged_model)

In [52]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlopsbatch/1', creation_time=1718746272105, experiment_id='1', last_update_time=1718746272105, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [53]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [60]:
# In this case we dont have train and test data, just the data to which the model will be applied. 
def load_model(run_id):
    logged_model =f's3://mlopsbatch/1/{RUN_ID}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    

    df_result.to_parquet(output_file, index=False)

In [61]:
apply_model(input_file=INPUT_FILE, run_id=RUN_ID, output_file=OUTPUT_FILE)

c:\Users\CamiloA\OneDrive\Documents\Python Scripts\mlops_2024\.venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\CamiloA\OneDrive\Documents\Python Scripts\mlops_2024\.venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\CamiloA\OneDrive\Documents\Python Scripts\mlops_2024\.venv\Lib\site-packages\sklearn\base.py:376: Inconsis

In [41]:
!DIR output/green/

Invalid switch - "green".
